In [35]:
import pandas as pd 
import matplotlib.pyplot as plt 

import os 
from glob import glob 
from tqdm import tqdm
import random
import warnings
warnings.filterwarnings("ignore")

import pandas as pd 
import matplotlib.pyplot as plt

from utils.indicators import movingAverageConverganceDivergance as macd 
from utils.indicators import relativeStrengthIndex as rsi 
from utils.indicators import averageTrueRange as atr 
from utils.indicators import bollingerBands as bbands 
from utils.indicators import stochasticIndicator as sti

from utils.strategies import BollingerBandsStrategy 
from utils.strategies import MACDStrategy 
from utils.strategies import BuyAndHoldStrategy 
from utils.strategies import SimpleMovingAverageStrategy

from backtesting import Backtest, Strategy 
from backtesting.lib import crossover, barssince, SignalStrategy, TrailingStrategy

In [42]:
stock_name = None 
random_seed = None
data_folder = "./data/raw/insample"
training = False 


In [43]:
if not training: 
    raise Exception("No need to run this cell")

# complete Insample Test
stock_name =None
stock_names = [] 
if stock_name is not None: 
    stock_names.append(f'{data_folder}/history_stock_{stock_name}.csv') 
else: 
    stock_names = glob(f'{data_folder}/*') 

all_trades = None
profit_buy_and_hold = 0 
profit_strategy = 0 

opt_params = {
    "ticker": [],
    "bb_window" : [] ,
    "rsi_window" : [],
    "rsi_smooth_window" : [],
    "rsi_upper_thres" : [] ,
    "rsi_lower_thres" : [] , 
    "macd_fast_ma_length" : [], 
    "macd_slow_ma_length" : [], 
    "macd_signal_ma_length": []
}


if not os.path.exists("./results/plots/optim"): 
    os.makedirs("./results/plots/optim", exist_ok=True) 

for stock_name in tqdm(stock_names): 
    stock_df = pd.read_csv(stock_name, index_col=False)

    bt = Backtest(stock_df, BuyAndHoldStrategy, cash=10_000) 
    stats = bt.run()
    profit_buy_and_hold += stats['Equity Final [$]'] - 10000 

    bt = Backtest(stock_df, BollingerBandsStrategy, cash = 10_000) 
    stats_bb = bt.run() 
    stats_bb, heatmap_bb = bt.optimize(
        bb_window = [14,21,30] ,
        rsi_window = [6,14,18],
        rsi_smooth_window = [3,5],
        rsi_upper_thres = [70,75,80] ,
        rsi_lower_thres = [20,25,30] ,
        constraint = lambda p: p.rsi_window >= 2*p.rsi_smooth_window,
        return_heatmap= True, 
        random_state=0, 
        maximize='SQN'
    )
    params = list(heatmap_bb.sort_values().iloc[-1:].index.values)[0] 
    opt_params['ticker'].append(stock_name.split('\\')[-1].split('.')[0].split('_')[-1])
    opt_params['bb_window'].append(params[0])
    opt_params['rsi_window'].append(params[1])
    opt_params['rsi_smooth_window'].append(params[2])
    opt_params['rsi_upper_thres'].append(params[3])
    opt_params['rsi_lower_thres'].append(params[4])


    bt = Backtest(stock_df, MACDStrategy, cash=10000) 
    stats_macd = bt.run() 
    stats_macd, heatmap_macd = bt.optimize(macd_fast_ma_length = list(range(12,30,2)), 
        macd_slow_ma_length = list(range(26,50,2)),
        macd_signal_ma_length = list(range(9,21,2)), 
        constraint = lambda p: p.macd_fast_ma_length*2<p.macd_slow_ma_length and p.macd_fast_ma_length > p.macd_signal_ma_length, 
        return_heatmap= True, 
        random_state=0, 
        maximize='SQN'
    )
    params = list(heatmap_macd.sort_values().iloc[-1:].index.values)[0] 
    opt_params['macd_fast_ma_length'].append(params[0])
    opt_params['macd_slow_ma_length'].append(params[1])
    opt_params['macd_signal_ma_length'].append(params[2])


df = pd.DataFrame(opt_params)
df.to_csv("./data/opt_params.csv", index = False)


Exception: No need to run this cell

Loading Opt Param File 

In [47]:
opt_param = pd.read_csv("./data/opt_params_bb_macd.csv", index_col = False)
opt_param.head()

,ticker,bb_window,rsi_window,rsi_smooth_window,rsi_upper_thres,rsi_lower_thres,macd_fast_ma_length,macd_slow_ma_length,macd_signal_ma_length
0,sh-600006,30,18,5,80,20,12,46,9
1,sh-600008,30,18,5,80,20,12,26,9
2,sh-600021,30,18,5,80,20,12,34,9
3,sh-600022,30,18,5,80,20,20,44,13
4,sh-600026,30,18,5,80,30,12,30,11
